In [1]:
import json

!pip uninstall -y trl peft transformers accelerate

!pip install -q trl==0.8.6 peft==0.10.0 transformers==4.40.0 accelerate==0.29.3 bitsandbytes

print("✅ Installation complete. NOW GO TO 'RUNTIME' > 'RESTART SESSION'.")

Found existing installation: trl 0.8.6
Uninstalling trl-0.8.6:
  Successfully uninstalled trl-0.8.6
Found existing installation: peft 0.10.0
Uninstalling peft-0.10.0:
  Successfully uninstalled peft-0.10.0
Found existing installation: transformers 4.40.0
Uninstalling transformers-4.40.0:
  Successfully uninstalled transformers-4.40.0
Found existing installation: accelerate 0.29.3
Uninstalling accelerate-0.29.3:
  Successfully uninstalled accelerate-0.29.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 5.1.2 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.40.0 which is incompatible.
✅ Installation complete. NOW GO TO 'RUNTIME' > 'RESTART SESSION'.


In [1]:
import json

data = [

    {"input": "Egg, Onion, Oil, Pepper", "output": "Spicy Omelet: 1. Chop onions and chilies. 2. Whisk eggs with salt and pepper. 3. Fry in oil until golden."},
    {"input": "Bread, Butter, Jam", "output": "Jam Toast: 1. Toast bread slices. 2. Spread butter while warm. 3. Top with fruit jam."},
    {"input": "Oats, Milk, Honey, Fruits", "output": "Oatmeal Bowl: 1. Boil milk. 2. Add oats and cook for 5 mins. 3. Top with honey and cut fruits."},
    {"input": "Flour, Milk, Egg, Sugar", "output": "Pancakes: 1. Mix flour, milk, egg, and sugar into a batter. 2. Pour onto a hot pan. 3. Flip when bubbles appear."},
    {"input": "Bread, Egg, Milk, Sugar", "output": "French Toast: 1. Whisk egg, milk, and sugar. 2. Dip bread in mixture. 3. Fry in butter until brown."},


    {"input": "Rice, Water, Salt", "output": "Steamed Rice: 1. Wash rice. 2. Add 2 cups water for 1 cup rice. 3. Boil and simmer for 20 mins."},
    {"input": "Chicken, Yogurt, Spices, Onion", "output": "Chicken Curry: 1. Marinate chicken in yogurt/spices. 2. Sauté onions. 3. Add chicken and cook until tender."},
    {"input": "Pasta, Tomato Sauce, Cheese, Basil", "output": "Tomato Pasta: 1. Boil pasta. 2. Heat sauce in a pan. 3. Toss pasta in sauce. 4. Top with cheese and basil."},
    {"input": "Rice, Egg, Soy Sauce, Peas, Carrots", "output": "Fried Rice: 1. Scramble eggs. 2. Sauté veggies. 3. Add cooked rice and soy sauce. 4. Mix well."},
    {"input": "Chicken, Lettuce, Caesar Dressing, Croutons", "output": "Chicken Caesar Salad: 1. Grill chicken slices. 2. Toss lettuce with dressing. 3. Top with chicken and croutons."},
    {"input": "Potato, Oil, Salt", "output": "French Fries: 1. Cut potatoes into strips. 2. Soak in cold water. 3. Deep fry in hot oil until crispy. 4. Sprinkle salt."},
    {"input": "Fish, Lemon, Butter, Garlic", "output": "Lemon Butter Fish: 1. Melt butter with garlic. 2. Pan-sear fish fillets. 3. Drizzle with lemon juice before serving."},
    {"input": "Burger Bun, Patty, Lettuce, Cheese, Tomato", "output": "Cheeseburger: 1. Grill the patty. 2. Toast the bun. 3. Assemble with lettuce, tomato, patty, and cheese."},
    {"input": "Pizza Dough, Tomato Sauce, Mozzarella, Pepperoni", "output": "Pepperoni Pizza: 1. Roll out dough. 2. Spread sauce and cheese. 3. Add pepperoni. 4. Bake at 400°F for 15 mins."},


    {"input": "Avocado, Onion, Tomato, Lime", "output": "Guacamole: 1. Mash avocado. 2. Mix in chopped onion and tomato. 3. Squeeze lime juice and salt."},
    {"input": "Bread, Cheese, Butter", "output": "Grilled Cheese Sandwich: 1. Place cheese between bread slices. 2. Butter the outside. 3. Grill until cheese melts."},
    {"input": "Tomato, Cucumber, Onion, Olive Oil", "output": "Garden Salad: 1. Chop all vegetables. 2. Toss in a bowl. 3. Drizzle with olive oil and salt."},
    {"input": "Potato, Mayo, Mustard, Onion", "output": "Potato Salad: 1. Boil and cube potatoes. 2. Mix with mayo, mustard, and onion. 3. Chill before serving."},
    {"input": "Chickpeas, Tahini, Lemon, Garlic, Olive Oil", "output": "Hummus: 1. Blend chickpeas, tahini, garlic, and lemon. 2. Add olive oil until smooth."},


    {"input": "Milk, Sugar, Tea Powder, Water", "output": "Masala Chai: 1. Boil water with tea powder. 2. Add milk and sugar. 3. Simmer and strain."},
    {"input": "Banana, Milk, Honey, Ice", "output": "Banana Smoothie: 1. Peel banana. 2. Blend with milk, honey, and ice until smooth."},
    {"input": "Coffee Powder, Water, Milk, Sugar", "output": "Coffee: 1. Boil water and milk. 2. Stir in coffee powder and sugar. 3. Serve hot."},
    {"input": "Lemon, Water, Sugar, Mint", "output": "Fresh Lemonade: 1. Squeeze lemon juice into water. 2. Stir in sugar. 3. Garnish with mint leaves."},
    {"input": "Mango, Yogurt, Sugar, Cardamom", "output": "Mango Lassi: 1. Blend mango pulp with yogurt. 2. Add sugar and cardamom. 3. Serve chilled."}
]
with open("recipes.json", "w") as f:
    json.dump(data, f, indent=4)

print("✅ Dataset 'recipes.json' created with sample recipes.")

✅ Dataset 'recipes.json' created with sample recipes.


In [2]:
import os
import torch
from datasets import Dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments
)
from peft import LoraConfig
from trl import SFTTrainer

os.environ["WANDB_DISABLED"] = "true"

data = [
    {"input": "Egg, Onion, Oil", "output": "Simple Omelet: 1. Chop onions. 2. Beat eggs. 3. Fry in oil."},
    {"input": "Bread, Butter, Jam", "output": "Jam Toast: 1. Toast bread. 2. Spread butter and jam."},
    {"input": "Chicken, Yogurt, Spices", "output": "Chicken Curry: 1. Marinate chicken. 2. Cook until tender."},
    {"input": "Rice, Water, Salt", "output": "Steamed Rice: 1. Boil water. 2. Add rice and salt. 3. Cook 20 mins."},
    {"input": "Milk, Sugar, Tea", "output": "Tea: 1. Boil water/milk. 2. Add tea and sugar. 3. Strain."}
]
dataset = Dataset.from_list(data)

def format_prompt(batch):
    output_texts = []
    for i in range(len(batch['input'])):
        text = f"User: Suggest a recipe using {batch['input'][i]}\nAssistant: {batch['output'][i]}"
        output_texts.append(text)
    return output_texts

model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"
)
model.config.use_cache = False

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

peft_config = LoraConfig(
    r=8, lora_alpha=16, lora_dropout=0.05, bias="none", task_type="CAUSAL_LM"
)

training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    max_steps=10,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=1,
    report_to="none",
    save_strategy="no"
)

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    formatting_func=format_prompt,
    max_seq_length=512,
    packing=False,
    args=training_args
)

print("🚀 Starting Training...")
trainer.train()
print("✅ Fine-tuning complete!")

print("\n🍳 Testing the Bot...")
text = "User: Suggest a recipe using Egg, Onion\nAssistant: "
inputs = tokenizer(text, return_tensors="pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens=50, do_sample=True)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possibl

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/accelerate/accelerator.py:469: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
max_steps is given, it will override any value given in num_train_epochs


🚀 Starting Training...


Step,Training Loss
1,2.699500
2,2.740000
3,2.660100
4,2.597100
5,2.515300
6,2.435800
7,2.448400
8,2.396400
9,2.421900
10,2.368900


✅ Fine-tuning complete!

🍳 Testing the Bot...
User: Suggest a recipe using Egg, Onion
Assistant: 🍳 Onion Rice Bowl - a healthy and flavorful egg bowl that you can easily prepare in 5 minutes! Ingredients:
- 1 cup uncooked rice
- 2 cups water


In [3]:
import os
import torch
from datasets import Dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

os.environ["WANDB_DISABLED"] = "true"
MODEL_NAME = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
OUTPUT_DIR = "recipe_bot_final"

print("🚀 INITIALIZING RECIPE BOT...")

data = [
    {"input": "Egg, Onion, Oil", "output": "Simple Omelet: 1. Chop onions. 2. Beat eggs. 3. Fry in oil."},
    {"input": "Bread, Butter, Jam", "output": "Jam Toast: 1. Toast bread. 2. Spread butter and jam."},
    {"input": "Chicken, Yogurt, Spices", "output": "Chicken Curry: 1. Marinate chicken. 2. Cook until tender."},
    {"input": "Rice, Water, Salt", "output": "Steamed Rice: 1. Boil water. 2. Add rice and salt. 3. Cook 20 mins."},
    {"input": "Milk, Sugar, Tea", "output": "Tea: 1. Boil water/milk. 2. Add tea and sugar. 3. Strain."}
]
dataset = Dataset.from_list(data)

def format_prompt(batch):
    output_texts = []
    for i in range(len(batch['input'])):
        text = f"User: Suggest a recipe using {batch['input'][i]}\nAssistant: {batch['output'][i]}"
        output_texts.append(text)
    return output_texts

print("⏳ Loading TinyLlama (this may take a minute)...")
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto"
)
model.config.use_cache = False

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

peft_config = LoraConfig(
    r=8, lora_alpha=16, lora_dropout=0.05, bias="none", task_type="CAUSAL_LM"
)

training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    max_steps=10,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=1,
    report_to="none",
    save_strategy="no"
)

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    formatting_func=format_prompt,
    max_seq_length=512,
    packing=False,
    args=training_args
)

print("🍳 Cooking up the model (Training)...")
trainer.train()
print("✅ Training Complete!")

print(f"💾 Saving to folder '{OUTPUT_DIR}'...")
trainer.model.save_pretrained(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)

print("\n" + "="*40)
print("🤖 RECIPE BOT INTERFACE")
print("="*40)

def get_recipe(ingredients):
    prompt = f"User: Suggest a recipe using {ingredients}\nAssistant: "
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")


    outputs = model.generate(
        **inputs,
        max_new_tokens=100,
        do_sample=True,
        temperature=0.7
    )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    if "Assistant: " in response:
        return response.split("Assistant: ")[1].strip()
    return response

while True:
    user_input = input("\nEnter Ingredients (or type 'exit'): ")
    if user_input.lower() in ['exit', 'quit']:
        print("Goodbye!")
        break

    print("👩‍🍳 Chef is thinking...")
    try:
        recipe = get_recipe(user_input)
        print(f"\n✨ Here is your recipe:\n{recipe}")
        print("-" * 30)
    except Exception as e:
        print(f"❌ Error: {e}")

🚀 INITIALIZING RECIPE BOT...
⏳ Loading TinyLlama (this may take a minute)...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Map:   0%|          | 0/5 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/accelerate/accelerator.py:469: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
max_steps is given, it will override any value given in num_train_epochs


🍳 Cooking up the model (Training)...


Step,Training Loss
1,2.699500
2,2.742000
3,2.664300
4,2.603000
5,2.523300
6,2.443400
7,2.457400
8,2.406000
9,2.431000
10,2.377600


✅ Training Complete!
💾 Saving to folder 'recipe_bot_final'...

🤖 RECIPE BOT INTERFACE

Enter Ingredients (or type 'exit'): exit
Goodbye!


In [4]:
from fastapi import FastAPI
from pydantic import BaseModel
import uvicorn
import nest_asyncio
import threading

app = FastAPI()

class RecipeRequest(BaseModel):
    ingredients: str

@app.post("/generate_recipe")
async def generate_recipe_endpoint(request: RecipeRequest):
    recipe = get_recipe(request.ingredients)
    return {"recipe": recipe}

@app.get("/")
async def root():
    return {"message": "Recipe Bot API is Running!"}

nest_asyncio.apply()

def run_server():
    uvicorn.run(app, host="127.0.0.1", port=8000)

thread = threading.Thread(target=run_server)
thread.start()

print("✅ API Server started successfully at http://127.0.0.1:8000")

✅ API Server started successfully at http://127.0.0.1:8000


In [5]:
import requests
import time
import re

def format_recipe_text(text):

    formatted = re.sub(r'(\d+\.)', r'\n\1', text)
    return formatted.strip()

print("\n" + "="*40)
print("🍳 WELCOME TO RECIPE BOT (Formatted)")
print("="*40)
print("Type 'exit' to quit.\n")

while True:
    user_input = input("Enter Ingredients (e.g., Egg, Onion): ")

    if user_input.lower() in ['exit', 'quit']:
        print("Goodbye!")
        break

    try:

        response = requests.post(
            "http://127.0.0.1:8000/generate_recipe",
            json={"ingredients": user_input}
        )

        if response.status_code == 200:
            raw_recipe = response.json().get("recipe")


            pretty_recipe = format_recipe_text(raw_recipe)

            print(f"\n🤖 Chef Bot says:")
            print("-" * 20)
            print(pretty_recipe)
            print("-" * 20 + "\n")
        else:
            print("❌ Error from API:", response.text)

    except Exception as e:
        print(f"❌ Connection Error: {e}")
        print("Make sure the Server Cell is still running!")


🍳 WELCOME TO RECIPE BOT (Formatted)
Type 'exit' to quit.

Enter Ingredients (e.g., Egg, Onion): pizza dough,water,tomato sauce
INFO:     127.0.0.1:38838 - "POST /generate_recipe HTTP/1.1" 200 OK

🤖 Chef Bot says:
--------------------
1. Preheat the oven to 375°F. 
2. Roll out the pizza dough. 
3. Spread tomato sauce on the pizza dough. 
4. Add shredded mozzarella cheese, vegetables, and meat, and bake according to the recipe. 
5. Remove from oven and let it cool for 5 minutes before slicing and serving. How to Make Homem
--------------------

Enter Ingredients (e.g., Egg, Onion): exit
Goodbye!
